In [1]:
import h5py
import os
import argparse
import torch
import matplotlib.pyplot as plt
import time
import datetime
import itertools
import math

import os.path as osp
import random
import pickle
import warnings

import glob
import h5py
import numpy as np

import torch.utils.data as data
import torch.nn.functional as F
import torch.distributed as dist
from torchvision.datasets.video_utils import VideoClips
import pytorch_lightning as pl

# from videogpt import VideoData, VideoGPT, load_videogpt
from videogpt import VideoGPT, load_videogpt
from videogpt.utils import save_video_grid

/home/groups/abrandt/SF_pytorch_1_9/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
def getNP(tensor, idx):
    output = tensor[idx,:,:,:,:]
    output = output.permute(1,2,3,0)
    output = output.cpu()
    return output.numpy()

In [3]:
def preprocess(video, resolution, sequence_length=None):
    # video: THWC, {0, ..., 255}
    video = video.permute(0, 3, 1, 2).float() / 255. # TCHW
    t, c, h, w = video.shape

    # temporal crop
    if sequence_length is not None:
        assert sequence_length <= t
        video = video[:sequence_length]

    # scale shorter side to resolution
    scale = resolution / min(h, w)
    if h < w:
        target_size = (resolution, math.ceil(w * scale))
    else:
        target_size = (math.ceil(h * scale), resolution)
    video = F.interpolate(video, size=target_size, mode='bilinear',
                          align_corners=False)

    # center crop
    t, c, h, w = video.shape
    w_start = (w - resolution) // 2
    h_start = (h - resolution) // 2
    video = video[:, :, h_start:h_start + resolution, w_start:w_start + resolution]
    video = video.permute(1, 0, 2, 3).contiguous() # CTHW

    video -= 0.5

    return video


class HDF5Dataset:
    """ Generic dataset for data stored in h5py as uint8 numpy arrays.
    Reads videos in {0, ..., 255} and returns in range [-0.5, 0.5] """
    def __init__(self, data_file, sequence_length, train=False, resolution=64):
        """
        Args:
            data_file: path to the pickled data file with the
                following format:
                {
                    'train_data': [B, H, W, 3] np.uint8,
                    'train_idx': [B], np.int64 (start indexes for each video)
                    'test_data': [B', H, W, 3] np.uint8,
                    'test_idx': [B'], np.int64
                }
            sequence_length: length of extracted video sequences
        """
        super().__init__()
        self.train = train
        self.sequence_length = sequence_length
        self.resolution = resolution

        # read in data
        self.data_file = data_file
        self.data = h5py.File(data_file, 'r')
        self.prefix = 'train' if train else 'test'
        self._images = self.data[f'{self.prefix}_data']
        self._idx = self.data[f'{self.prefix}_idx']
        self.size = len(self._idx)

    def setstate(self, state):
        self.__dict__ = state
        self.data = h5py.File(self.data_file, 'r')
        self._images = self.data[f'{self.prefix}_data']
        self._idx = self.data[f'{self.prefix}_idx']

    def getitem(self, idx):
        start = self._idx[idx]
        end = self._idx[idx + 1] if idx < len(self._idx) - 1 else len(self._images)
        assert end - start >= 0
        start = start + np.random.randint(low=0, high=end - start - self.sequence_length + 1) #Andea added +1 for inclusive of endpoint
        assert start < start + self.sequence_length <= end
        video = torch.tensor(self._images[start:start + self.sequence_length])
        return dict(video=preprocess(video, self.resolution), idx=idx)
#         return dict(video=preprocess(video, self.resolution))


In [4]:
def getSample(ckpt_path, n, start):
    
    batch = {}
    if not os.path.exists(ckpt_path):
        gpt = load_videogpt(ckpt_path)
    else:
        gpt = VideoGPT.load_from_checkpoint(ckpt_path)
    gpt = gpt.cuda()
    gpt.eval()

    data_file = 'GPT_full_2min_2019_test_data.hdf5'
    #data_file = 'GPT_benchmark.hdf5'
    sequence_length = 16
    train=False
    resolution=64


    data = HDF5Dataset(data_file, sequence_length, train, resolution)
    tempBatch = torch.empty((n, 3, 16, 64, 64), device = 'cuda')

    for pos, example in enumerate(range(start, start+n)):
        item = data.getitem(example)
        tempBatch[pos] = item['video']

    batch['video'] = tempBatch

    samples = gpt.sample(n, batch)
    return samples

In [5]:
def runIteration(ckpt_path, num_test, n, it, save_folder):
    all_test_samples = np.empty([num_test, 16, 64, 64, 3]) #initialize a holder for the sample results (diff from sample shape)

    step = n
    for batch_start in range(0, num_test, step):

        start = batch_start

        #check for partial batch at end
        if (start + step) > num_test:
            n = num_test - start

        output = getSample(ckpt_path, n, start)
        output = output.permute(0,2,3,4,1) #original shape is (num_examples, 3, 16, 64, 64)
        output = output.cpu()
        output = output.numpy()
        all_test_samples[start:(start+n)] = output
    filename = save_folder + "/sample_" + str(it) + ".npy"
    np.save(filename, all_test_samples)

In [6]:
num_test = 2582
#num_test = 2
n = 50 # num videos from the test (2582 is all test videos)
ckpt_path = '/scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt'

In [7]:
save_folder = "/scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/PhyGPT/inference/PhyGPT4x4x4_2019_test"
if os.path.isdir(save_folder)==False:
    os.makedirs(save_folder)
print(save_folder)

/scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/PhyGPT/inference/PhyGPT4x4x4_2019_test


In [8]:
num_it = 50
for it in range(45, num_it):
    runIteration(ckpt_path, num_test, n, it, save_folder)
    print('Iteration ', it, 'complete!')

/home/users/ynie/.local/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/migration.py:200: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN

Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/trained_models/VQVAE_full_2min/lightning_logs/version_0/checkpoints/epoch=47-step=160031.ckpt`
100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.18it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/t

Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/trained_models/VQVAE_full_2min/lightning_logs/version_0/checkpoints/epoch=47-step=160031.ckpt`
100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.15it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/t

Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/trained_models/VQVAE_full_2min/lightning_logs/version_0/checkpoints/epoch=47-step=160031.ckpt`
100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.16it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/t

Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/trained_models/VQVAE_full_2min/lightning_logs/version_0/checkpoints/epoch=47-step=160031.ckpt`
100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.17it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/t

...

100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.18it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/trained_models/VQVAE_full_2min/lightning_logs/version_0/checkpoints/epoch=47-step=160031.ckpt`
100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.18it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to you

100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.23it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/trained_models/VQVAE_full_2min/lightning_logs/version_0/checkpoints/epoch=47-step=160031.ckpt`
100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.21it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to you

100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.20it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/trained_models/VQVAE_full_2min/lightning_logs/version_0/checkpoints/epoch=47-step=160031.ckpt`
100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.23it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to you

100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.23it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../VideoGPT/trained_models/PhyGPT_full_2min/lightning_logs/version_36584693/checkpoints/epoch=17-step=30005.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../GAN_project/models/VideoGPT/trained_models/VQVAE_full_2min/lightning_logs/version_0/checkpoints/epoch=47-step=160031.ckpt`
100%|██████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [00:33<00:00, 30.22it/s]
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.0. To apply the upgrade to you

Iteration  49 complete!
